In [0]:
"""
This notebook monitors a specified metric in a given table for threshold violations across multiple evaluation windows.

Parameters:
    - table_name_under_monitor (str): Full three-level name of the table to monitor.
    - metric_to_monitor (str): Name of the metric to check for threshold violation.
    - metric_violation_threshold (str): Threshold value for the metric.
    - metric_violation_operator (str): Operator for threshold comparison (>, <, >=, <=).
    - num_evaluation_windows (str): Number of windows to evaluate for violations.
    - num_violation_windows (str): Number of windows that must violate the threshold to trigger an alert.

Workflow:
    1. Parameters are set via Databricks widgets.
    2. A SQL query is dynamically generated to check for metric violations.
    3. The query is executed and results are displayed.
    4. The violation status can be set as a job task value for downstream workflows.

Intended Use:
    - Automated monitoring of ML inference tables for metric threshold violations.
    - Integration with Databricks Jobs for alerting and workflow orchestration.
"""

In [0]:
# List of input args needed to run the notebook as a job.
# Provide them via DB widgets or notebook arguments.
#
# Name of the table that is currently being monitored
dbutils.widgets.text(
    "table_name_under_monitor", "mlops_dbx_talk_dev.churn.telco_churn_inference_table", label="Full (three-Level) table name"
)
# Metric to be used for threshold violation check
dbutils.widgets.text(
    "metric_to_monitor", "accuracy_score", label="Metric to be monitored for threshold violation"
)

# Threshold value to be checked
dbutils.widgets.text(
    "metric_violation_threshold", "0.8", label="Threshold value for metric violation"
)

# Threshold value to be checked
dbutils.widgets.text(
    "num_evaluation_windows", "5", label="Number of windows to check for violation"
)

# Threshold value to be checked
dbutils.widgets.text(
    "num_violation_windows", "2", label="Number of windows that need to violate the threshold"
)
dbutils.widgets.text(
    "metric_violation_operator", ">", label="Metric violation operator (>, <, >=, <=)"
)

In [0]:
import sys
sys.path.append('..')
from metric_violation_check_query import sql_query

table_name_under_monitor = dbutils.widgets.get("table_name_under_monitor")
metric_to_monitor = dbutils.widgets.get("metric_to_monitor")
metric_violation_threshold = dbutils.widgets.get("metric_violation_threshold")
num_evaluation_windows = dbutils.widgets.get("num_evaluation_windows")
num_violation_windows = dbutils.widgets.get("num_violation_windows")
metric_violation_operator = dbutils.widgets.get("metric_violation_operator")

formatted_sql_query = sql_query.format(
    table_name_under_monitor=table_name_under_monitor,
    metric_to_monitor=metric_to_monitor,
    metric_violation_threshold=metric_violation_threshold,
    metric_violation_operator=metric_violation_operator,
    num_evaluation_windows=num_evaluation_windows,
    num_violation_windows=num_violation_windows)
is_metric_violated = bool(spark.sql(formatted_sql_query).toPandas()["query_result"][0])

dbutils.jobs.taskValues.set("is_metric_violated", is_metric_violated)

